In [89]:
import os
import time
# spark imports
import findspark
findspark.init('C:/Users/Lenovo/Downloads/spark-3.0.0-preview2-bin-hadoop3.2')

import numpy as np
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [4]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [23]:
books = spark.read.load(r"C:\Users\Lenovo\Documents\Springboard\Capstone\Dataset\BX-CSV-Dump\BX-Books.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')

In [27]:
win_row_number = Window.orderBy("ISBN")
isbn = books.select(F.row_number().over(win_row_number).alias("book_id"), "ISBN")
isbn.cache().show(3)

+-------+----------+
|book_id|      ISBN|
+-------+----------+
|      1|0000913154|
|      2|0001010565|
|      3|0001046438|
+-------+----------+
only showing top 3 rows



In [34]:
ratings = spark.read.load(r"C:\Users\Lenovo\Documents\Springboard\Capstone\Dataset\BX-CSV-Dump\BX-Book-Ratings.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')
ratings = ratings.withColumn("Book-Rating", ratings["Book-Rating"].cast("Float"))
ratings_with_bid = ratings.join(isbn, on=["ISBN"], how="inner").select(F.col("User-ID").alias("userID"), F.col("book_id").alias("productID"), F.col("Book-Rating").alias("rating")).orderBy("userID")
ratings_with_bid.show(3)

+------+---------+------+
|userID|productID|rating|
+------+---------+------+
|     2|    25031|   0.0|
|     8|    60204|   0.0|
|     8|     8213|   0.0|
+------+---------+------+
only showing top 3 rows



In [73]:
ratings_with_bid.select(F.max(F.col("productID"))).show()

+--------------+
|max(productID)|
+--------------+
|        271379|
+--------------+



In [70]:
ratings_with_bid.groupBy("userID").pivot("productID")

AnalysisException: The pivot column productID has more than 10000 distinct values, this could indicate an error. If this was intended, set spark.sql.pivotMaxValues to at least the number of distinct values of the pivot column.;

In [81]:
limited_ratings = ratings_with_bid.limit(20).groupBy("userID").pivot("productID").count()

In [85]:
limited_ratings.show(5, truncate=True)

+------+----+----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+
|userID|  74|8213|25031|60204|71717|77840|82576|87557|103718|141588|143543|157823|166788|177289|203483|219486|222614|227878|231749|242501|
+------+----+----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+
|     2|null|null|    1| null| null| null| null| null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|
|     8|   1|   1| null|    1|    1|    1|    1| null|  null|     1|     1|     1|     1|     1|     1|     1|     1|     1|     1|     1|
|     9|null|null| null| null| null| null| null|    1|     1|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|
+------+----+----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+



In [86]:
mat = RowMatrix(limited_ratings.rdd)

In [88]:
svd = mat.computeSVD(20, computeU=True)

Py4JJavaError: An error occurred while calling o527.computeSVD.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 68.0 failed 1 times, most recent failure: Lost task 0.0 in stage 68.0 (TID 69, DESKTOP-AI4EQ38, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Lenovo\Downloads\spark-3.0.0-preview2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 597, in main
  File "C:\Users\Lenovo\Downloads\spark-3.0.0-preview2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 589, in process
  File "C:\Users\Lenovo\Downloads\spark-3.0.0-preview2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 513, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\Lenovo\Downloads\spark-3.0.0-preview2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "C:\Users\Lenovo\Downloads\spark-3.

In [99]:
arr = []
new_limited_ratings_rdd = limited_ratings.rdd.map(lambda x: np.append(arr, x))

In [100]:
new_limited_ratings_rdd.take(3)

[array([2, None, None, 1, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None], dtype=object),
 array([8, 1, 1, None, 1, 1, 1, 1, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1], dtype=object),
 array([9, None, None, None, None, None, None, None, 1, 1, None, None,
        None, None, None, None, None, None, None, None, None], dtype=object)]

In [101]:
mat = RowMatrix(new_limited_ratings_rdd)

In [102]:
svd = mat.computeSVD(20, computeU=True)

Py4JJavaError: An error occurred while calling o774.computeSVD.
: breeze.linalg.NotConvergedException: 
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:120)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:46)
	at breeze.generic.UFunc.apply$(UFunc.scala:45)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computeSVD(RowMatrix.scala:362)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computeSVD(RowMatrix.scala:296)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at

In [33]:
rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(5, computeU=True)

In [6]:
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

In [10]:
rows.collect()

[SparseVector(5, {1: 1.0, 3: 7.0}),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [12]:
U.rows.collect()

[DenseVector([-0.3883, -0.9198, -0.0564, 0.0, 0.0]),
 DenseVector([-0.5302, 0.273, -0.8027, 0.0, 0.0]),
 DenseVector([-0.7538, 0.2818, 0.5937, 0.0, 0.0])]

In [15]:
s

DenseVector([13.0293, 5.3686, 2.533, 0.0, 0.0])

In [22]:
V

DenseMatrix(5, 5, [-0.3128, -0.0298, -0.1221, -0.7185, -0.6084, 0.3117, -0.1713, 0.1526, ..., 0.2383, -0.0209, -0.4626, -0.0745, 0.9735, -0.0345, -0.1391, 0.1618], 0)